In [ ]:
from keras.datasets import mnist
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import binarize
from skimage.filters import threshold_local
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import cv2
from scipy.ndimage import gaussian_filter

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

max_samples_per_class = 3000

selected_indices = []
for class_label in np.unique(y_train):
    indices = np.where(y_train == class_label)[0]
    num_samples = min(max_samples_per_class, len(indices))
    selected_indices.extend(indices[:num_samples])

x_train = x_train[selected_indices]
y_train = y_train[selected_indices]
x_train = x_train.reshape((-1, 28*28))
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,
                                                  test_size=0.2, random_state=42, stratify=y_train)

unique_classes, counts = np.unique(y_train, return_counts=True)

#for i, label in enumerate(unique_classes):
#   print(f"Osztály {label}: {counts[i]} darab minta")

In [ ]:
clf = MLPClassifier(solver='adam', activation='tanh', hidden_layer_sizes=(128,128),
                    max_iter=1, learning_rate='adaptive')

train_accuracies = []
val_accuracies = []

for i in range(85):
    clf.partial_fit(x_train, y_train, classes=np.unique(y_train))

    ytrain_pred = clf.predict(x_train)
    train_accuracy = accuracy_score(y_train, ytrain_pred)
    train_accuracies.append(train_accuracy)

    yval_pred = clf.predict(x_val)
    val_accuracy = accuracy_score(y_val, yval_pred)
    val_accuracies.append(val_accuracy)

plt.plot(train_accuracies, label='Tanító adathalmaz')
plt.plot(val_accuracies, label='Validációs adathalmaz')
plt.xlabel('Iterációk')
plt.ylabel('Pontosság')
plt.legend()
plt.show()


In [ ]:
def image_to_bin(img):
    data = list(img.getdata())
    for i in range(len(data)):
        data[i] = 255 - data[i]
    data = np.array(data)/256.0
    return data

In [ ]:
image_folder = "drawn_digits"
images = []
predictions = []

for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = Image.open(file_path)
        binimg = image_to_bin(image)
        p = clf.predict([binimg])
        images.append(image)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='gray')
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()

In [ ]:
image_folder = "handwritten_digits"
images = []
predictions = []

for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = Image.open(file_path)
        binimg = image_to_bin(image)
        p = clf.predict([binimg])
        images.append(image)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='gray')
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()

# Zajszűrés

In [ ]:
def noise_reduction(image_array, sigma):
    return gaussian_filter(image_array, sigma)

# Képek beolvasása és osztályozás végrehajtása
for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = Image.open(file_path)
        
        # Zajszűrés a scipy.ndimage Gaussian szűrőjével
        denoised_image = noise_reduction(np.array(image), sigma=155.0)

        # Átalakítjuk a képet szürkeárnyalatossá (grayscale)
        denoised_image_gray = np.array(Image.fromarray(denoised_image).convert("L"))

        p = clf.predict([denoised_image_gray.flatten()])
        images.append(image)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='gray')
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()


## küszöbölés és zajszűrés

In [ ]:
def noise_reduction(image, kernel_size):
    blurred_image = cv2.GaussianBlur(image, (kernel_size, kernel_size), 1)
    return blurred_image

def local_thresholding(img):
    block_size = 35
    local_threshold = threshold_local(img, block_size, offset=30, method='gaussian')
    binary_image = (img > local_threshold).astype(np.uint8) * 255

    inverted_image = cv2.bitwise_not(binary_image)

    return inverted_image

image_folder = "handwritten_digits"
images = []
predictions = []

# Képek beolvasása és osztályozás végrehajtása
for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # Képek szürkeárnyalatosan beolvasása

        # Zajcsökkentés az OpenCV Gauss szűrőjével
        denoised_image = noise_reduction(image, kernel_size=1)

        # Lokális küszöbölés zajszűrést követően
        binimg = local_thresholding(denoised_image)

        p = clf.predict([binimg.flatten()])
        images.append(binimg)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='gray')
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()


# Gauss filter

In [ ]:
def preprocess_image(image):
    blurred_image = cv2.GaussianBlur(image, (1,1), 0)
    _, binary_image = cv2.threshold(blurred_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return binary_image

def contours(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 100)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_img = image.copy()
    cv2.drawContours(contour_img, contours, -1, (255,0, 255), 1)
    return contour_img

image_folder = "handwritten_digits"
images = []
predictions = []

for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

        processed_image = preprocess_image(image)

        image_with_contours = contours(cv2.cvtColor(processed_image, cv2.COLOR_GRAY2RGB))
        image_with_contours_gray = cv2.cvtColor(image_with_contours, cv2.COLOR_RGB2GRAY)

        p = clf.predict([image_with_contours_gray.flatten()])
        images.append(image_with_contours)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i])
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()


# Median filter

In [ ]:
def preprocess_image(image):
    denoised_image = cv2.medianBlur(image, 1)
    _, binary_image = cv2.threshold(denoised_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return binary_image

def contours(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 5, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_img = image.copy()
    cv2.drawContours(contour_img, contours, -1, (255,0, 255), 1)
    return contour_img

image_folder = "handwritten_digits"
images = []
predictions = []

for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

        processed_image = preprocess_image(image)
        image_with_contours = contours(cv2.cvtColor(processed_image, cv2.COLOR_GRAY2RGB))

        image_with_contours_gray = cv2.cvtColor(image_with_contours, cv2.COLOR_RGB2GRAY)

        p = clf.predict([image_with_contours_gray.flatten()])
        images.append(image_with_contours)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i])
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()

#  Bilaterális szűrő

In [ ]:
def preprocess_image(image):
    denoised_image =cv2.bilateralFilter(image, 3, 85, 9)
    _, binary_image = cv2.threshold(denoised_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return binary_image

def contours(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 100)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_img = image.copy()
    cv2.drawContours(contour_img, contours, -1, (255,0, 255), 1)
    return contour_img

image_folder = "handwritten_digits"
images = []
predictions = []

for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        processed_image = preprocess_image(image)

        image_with_contours = contours(cv2.cvtColor(processed_image, cv2.COLOR_GRAY2RGB))
        image_with_contours_gray = cv2.cvtColor(image_with_contours, cv2.COLOR_RGB2GRAY)

        p = clf.predict([image_with_contours_gray.flatten()])
        images.append(image_with_contours)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i])
    ax.set_title("Prediction: {}".format(predictions[i]))


plt.tight_layout()
plt.show()
